In [66]:
import pandas as pd
import CFS
import numpy as np

import sys
import traceback
import warnings
import os
import copy
import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [48]:
path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/old_datasets/CFS/'
projects = [join(path,f) for f in listdir(path) if isfile(join(path, f))]
def apply_cfs(df):
    y = df.bug.values
    X = df.drop(labels = ['bug'],axis = 1)
    X = X.values
    selected_cols = CFS.cfs(X,y)
    cols = df.columns[[selected_cols]].tolist()
    cols.append('bug')
    return df[cols],cols

In [70]:
project = {}
for proj in projects:
    df = pd.read_csv(proj)
    y = df.bug
    X = df.drop(labels = ['bug'],axis = 1)
    kf = StratifiedKFold(n_splits = 5)
    score = []
    for i in range(5):
        for train_index, tune_index in kf.split(X, y):
            X_train, X_tune = X.iloc[train_index], X.iloc[tune_index]
            y_train, y_tune = y[train_index], y[tune_index]
            _df = pd.concat([X_train,y_train], axis = 1)
            _df,selected_cols = apply_cfs(_df)
            y_train = _df.bug
            X_train = _df.drop(labels = ['bug'],axis = 1)
            _df_tune = pd.concat([X_tune,y_tune], axis = 1)
            _df_tune = _df_tune[selected_cols]
            y_tune = _df_tune.bug
            X_tune = _df_tune.drop(labels = ['bug'],axis = 1)
            clf = LogisticRegression()
            clf.fit(X_train,y_train)
            predicted = clf.predict(X_tune)
            score.append(round(metrics.f1_score(y_tune,predicted,average='weighted'),2))
            print(metrics.f1_score(y_tune,predicted,average='weighted'))
    project[proj.rsplit('/',1)[1].rsplit('.',1)[0]] = score

0.8449317038102085
0.8085834240398071
0.9247885491216656
0.8301404853128991
0.8849821485063496
0.8449317038102085
0.8085834240398071
0.9247885491216656
0.8301404853128991
0.8849821485063496
0.8449317038102085
0.8085834240398071
0.9247885491216656
0.8301404853128991
0.8849821485063496
0.8449317038102085
0.8085834240398071
0.9247885491216656
0.8301404853128991
0.8849821485063496
0.8449317038102085
0.8085834240398071
0.9247885491216656
0.8301404853128991
0.8849821485063496
0.7669565217391304
0.9084848484848485
0.84
0.84
0.7466666666666667
0.7669565217391304
0.9084848484848485
0.84
0.84
0.7466666666666667
0.7669565217391304
0.9084848484848485
0.84
0.84
0.7466666666666667
0.7669565217391304
0.9084848484848485
0.84
0.84
0.7466666666666667
0.7669565217391304
0.9084848484848485
0.84
0.84
0.7466666666666667
0.9352657004830918
0.9343210968597965
0.9343210968597965
0.9402985074626866
0.9554500226142018
0.9352657004830918
0.9343210968597965
0.9343210968597965
0.9402985074626866
0.9554500226142018


In [71]:
project

{'ant-1.5': [0.84,
  0.81,
  0.92,
  0.83,
  0.88,
  0.84,
  0.81,
  0.92,
  0.83,
  0.88,
  0.84,
  0.81,
  0.92,
  0.83,
  0.88,
  0.84,
  0.81,
  0.92,
  0.83,
  0.88,
  0.84,
  0.81,
  0.92,
  0.83,
  0.88],
 'ant-1.3': [0.77,
  0.91,
  0.84,
  0.84,
  0.75,
  0.77,
  0.91,
  0.84,
  0.84,
  0.75,
  0.77,
  0.91,
  0.84,
  0.84,
  0.75,
  0.77,
  0.91,
  0.84,
  0.84,
  0.75,
  0.77,
  0.91,
  0.84,
  0.84,
  0.75],
 'camel-1.0': [0.94,
  0.93,
  0.93,
  0.94,
  0.96,
  0.94,
  0.93,
  0.93,
  0.94,
  0.96,
  0.94,
  0.93,
  0.93,
  0.94,
  0.96,
  0.94,
  0.93,
  0.93,
  0.94,
  0.96,
  0.94,
  0.93,
  0.93,
  0.94,
  0.96],
 'camel-1.6': [0.72,
  0.75,
  0.75,
  0.74,
  0.76,
  0.72,
  0.75,
  0.75,
  0.74,
  0.76,
  0.72,
  0.75,
  0.75,
  0.74,
  0.76,
  0.72,
  0.75,
  0.75,
  0.74,
  0.76,
  0.72,
  0.75,
  0.75,
  0.74,
  0.76],
 'velocity-1.5': [0.74,
  0.73,
  0.79,
  0.79,
  0.73,
  0.74,
  0.73,
  0.79,
  0.79,
  0.73,
  0.74,
  0.73,
  0.79,
  0.79,
  0.73,
  0.74,
  0.

In [57]:
attr = {'ant-1.3': ['wmc','cbo','rfc','lcom','ce','loc','dam','cam','bug'],
        'ant-1.5': ['rfc','ce','lcom','cbm','bug'],
        'camel-1.0': ['cbo','lcom','ca','avg_cc','bug'],
        'camel-1.6': ['dit','noc','cbo','lcom','ca','cbm','amc','max_cc','bug'],
        'jedit-3.2': ['dit','rfc','lcom','ce','lcom3','loc','dam','moa','ic','cbm','max_cc','bug'],
        'jedit-4.2': ['rfc','ca','ce','npm','lcom3','moa','cam','cbm','max_cc','bug'],
        'log4j-1.0': ['wmc','cbo','rfc','lcom','ca','ce','npm','loc','dam','cam','avg_cc','bug'],
        'log4j-1.1': ['wmc','rfc','lcom','ce','npm','lcom3','moa','mfa','bug'],
        'velocity-1.4': ['dit','ca','npm','ic','cbm','bug'],
        'velocity-1.5': ['cbo','rfc','lcom3','loc','amc','bug']}

In [64]:
project1 = {}
for proj in projects:
    df = pd.read_csv(proj)
    name = proj.rsplit('/',1)[1].rsplit('.',1)[0]
    print(name)
    y = df.bug
    X = df.drop(labels = ['bug'],axis = 1)
    kf = StratifiedKFold(n_splits = 5)
    score = []
    for i in range(5):
        for train_index, tune_index in kf.split(X, y):
            X_train, X_tune = X.iloc[train_index], X.iloc[tune_index]
            y_train, y_tune = y[train_index], y[tune_index]
            _df = pd.concat([X_train,y_train], axis = 1)
            #_df,selected_cols = apply_cfs(df)
            selected_cols = attr[name]
            _df = df[selected_cols]
            y_train = _df.bug
            X_train = _df.drop(labels = ['bug'],axis = 1)
            _df_tune = pd.concat([X_tune,y_tune], axis = 1)
            _df_tune = _df_tune[selected_cols]
            y_tune = _df_tune.bug
            X_tune = _df_tune.drop(labels = ['bug'],axis = 1)
            clf = LogisticRegression()
            clf.fit(X_train,y_train)
            predicted = clf.predict(X_tune)
            score.append(round(metrics.f1_score(y_tune,predicted,average='weighted'),2))
            print(metrics.f1_score(y_tune,predicted,average='weighted'))
    project1[name] = score

ant-1.5
0.8316096747289409
0.9271932629783605
0.9461710703090012
0.8849821485063496
0.8849821485063496
ant-1.3
0.7669565217391304
0.8728239202657807
0.7880398671096346
0.9576079734219269
0.7669565217391304
camel-1.0
0.9352657004830918
0.9343210968597965
0.9343210968597965
0.9479320952489462
0.9554500226142018
camel-1.6
0.7176717083798065
0.734464431145201
0.7680155798508674
0.7568516042780749
0.7986332523096996
velocity-1.5
0.648534525583706
0.6676065869614256
0.787460815047022
0.7355425601039637
0.783625730994152
velocity-1.4
0.8589743589743589
0.8669467787114847
0.7818540433925049
0.7818540433925049
0.8584987057808454
jedit-4.2
0.8590355060943297
0.8590355060943297
0.8914177335229967
0.9133034773445734
0.8097014925373134
jedit-3.2
0.8181818181818182
0.7681818181818182
0.7408700712934673
0.7742448330683623
0.8427350427350427
log4j-1.1
0.8190993788819875
0.7674065065369414
0.8541114058355438
0.8445747800586509
0.9047619047619048
log4j-1.0
0.8571428571428571
0.8430335097001763
0.9215167

In [72]:
for keys in project:
    print(keys,np.median(project[keys]))

ant-1.5 0.84
ant-1.3 0.84
camel-1.0 0.94
camel-1.6 0.75
velocity-1.5 0.74
velocity-1.4 0.87
jedit-4.2 0.89
jedit-3.2 0.74
log4j-1.1 0.84
log4j-1.0 0.76


In [ ]:
{'ant-1.5': [0.84, 0.81, 0.92, 0.83, 0.88],
 'ant-1.3': [0.77, 0.91, 0.84, 0.84, 0.75],
 'camel-1.0': [0.94, 0.93, 0.93, 0.94, 0.96],
 'camel-1.6': [0.72, 0.75, 0.75, 0.74, 0.76],
 'velocity-1.5': [0.74, 0.73, 0.79, 0.79, 0.73],
 'velocity-1.4': [0.87, 0.9, 0.78, 0.8, 0.87],
 'jedit-4.2': [0.83, 0.92, 0.89, 0.91, 0.81],
 'jedit-3.2': [0.82, 0.77, 0.71, 0.74, 0.71],
 'log4j-1.1': [0.86, 0.64, 0.8, 0.84, 0.9],
 'log4j-1.0': [0.76, 0.8, 0.73, 0.83, 0.75]}